In [1]:
import pandas as pd
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df_recetas = pd.read_csv('recetas.csv', delimiter='|')

In [3]:
df_recetas = df_recetas.dropna(subset=['Tiempo'])

In [4]:
df_recetas = df_recetas.dropna(subset=['Tipo'])

In [5]:
df_recetas = df_recetas.dropna(subset=['Ingredientes'])

In [6]:
df_recetas = df_recetas.dropna(subset=['Dificultad'])


In [7]:
df_recetas = df_recetas.drop(columns = ['Num_comentarios', 'Num_reviews', 'Fecha_modificacion'])

In [8]:
df_recetas.head()

,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Ingredientes
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,"3 tazas de harina de trigo (420 gramos),1 pizc..."
5,74068,Recetas de Aperitivos y tapas,Receta de Hummus de zanahoria,NaN,baja,6.0,1h 30m,Entrante,https://www.recetasgratis.net/receta-de-hummus...,"2 zanahorias,1 chorro de aceite de olivia virg..."


# Scrapeo

In [9]:
driver = 'chromedriver.exe'
url = 'https://www.recetasgratis.net/'

In [10]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path=driver, options=options)

In [11]:
driver.get(url)

In [12]:
button = driver.find_element_by_xpath('/html/body/div[7]/div[2]/div[1]/div[2]/div[2]/button[1]/p')

In [13]:
button.click()

In [14]:
lista_id = []
pasos = []
ins = []
num = -1
for url in tqdm(df_recetas['Link_receta']):
    driver.get(url)
    num += 1
    for i in driver.find_elements_by_class_name('apartado'):
        if driver.find_elements_by_class_name('orden') != []:
            try:
                lista_id.append(df_recetas['Id'].iloc[[num]].values)
                pasos.append(int(i.find_element_by_tag_name('div').text))
                ins.append(i.find_element_by_tag_name('p').text)
                
            except:
                continue
    print(lista_id)
    print(pasos)
    print(ins)
    print('\n')
df_pasos = pd.DataFrame(list(zip(lista_id, pasos, ins)),
               columns =['Id', 'Paso', 'Instruccion'])

  0%|          | 0/11370 [00:00<?, ?it/s]

[array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64), array([74133], dtype=int64)]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
['Lavar y desinfectar los vegetales para empezar a preparar los tacos, así como los instrumentos de cocina. En una tabla, pica finamente la cebolla blanca y ralla el ajo. Reserva estos ingredientes.', 'Trocea las aceitunas con pimiento, pica el perejil y reserva. Corta la papa en cubos muy pequeños y cuécelo en una olla con agua hirviendo.', 'Mientras hierven las papas, calienta el aceite en una sartén amplia y sofríe la cebolla y el ajo hasta que estén transparentes. Acto seguido, añade la carne molida, separándola todo lo que sea posible.', 'Sofríe 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.93)


In [16]:
df_pasos

,Id,Paso,Instruccion
0,[74133],1,Lavar y desinfectar los vegetales para empezar...
1,[74133],2,"Trocea las aceitunas con pimiento, pica el per..."
2,[74133],3,"Mientras hierven las papas, calienta el aceite..."
3,[74133],4,Sofríe la carne hasta que adquiera un buen col...
4,[74133],5,"Pasados los 5 minutos, incorpora el puré de to..."
5,[74133],6,Agrega las especias a la carne y salpimienta a...
6,[74133],7,"Para este punto, las patatas ya deben estar co..."
7,[74133],8,Prepara dos onzas de tequila blanco en un vaso...
8,[74133],9,Reduce la temperatura y tapa la sartén. Mientr...
9,[74133],10,"Es una sartén o comal, fríe las tortillas con ..."


In [32]:
receta = driver.find_elements_by_class_name('apartado')

for i in receta:
    if i.find_elements_by_class_name('orden') != []:
        print("Columna pos", i.find_element_by_tag_name('div').text)
        print("Columna inst", i.find_element_by_tag_name('p').text)
        print(receta)

Columna pos 1
Columna inst Para comenzar a confitar tomates cherry, lava los tomates y sécalos. Si son grandes, pártelos por la mitad.
[<selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="0ec00acc-fccc-46a1-abc7-8de2e302035a")>, <selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="1ea10b30-88a6-4b3f-bc6c-1c15b6bfb6b5")>, <selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="aa010a16-927c-4b85-9af3-8a5c2dbba78f")>, <selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="99832823-b8ef-4637-87ca-fc72939e70b6")>, <selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="b0cc25be-c5c3-4d96-9385-67f0155cd306")>, <selenium.webdriver.remote.webelement.WebElement (session="894b21845c844e43a00504dc74c8c928", element="dec49738-45d1-4523-a2ee-d8cd87de7167")>, 

Columna pos 1
Columna inst Prepara los higaditos de pollo limpiándolos bien y sacando los nervios que puedan contener. Luego, córtalos en trozos medianos, corta la cebolla en juliana y lamina los ajos. Reserva.
Columna pos 2
Columna inst En una cacerola, vierte un buen chorro de aceite y añade la cebolla y el ajo con una pizca de sal. Cocina a fuego suave para que se reblandezca sin que llegue a dorarse demasiado, lo justo para darle un toque como de caramelizado.
Columna pos 3
Columna inst Una vez tengas pochada la cebollita con el ajo, aproximadamente en unos 4 minutos a fuego suave, añade los higaditos y remueve. Incorpora la hoja de laurel, la pimienta molida y dale un punto de sal si lo precisa.
Columna pos 4
Columna inst Sofríe los higaditos unos 3 minutos a fuego medio y vierte el vino blanco. Deja que evapore el alcohol 1-2 minutos. En total, deberás cocinar los higaditos de pollo con cebolla y vino blanco unos 5 minutos. En este paso, si lo deseas, puedes añadir alguna especia